TODO
Correction for SepReformerSettings sr duplicated

In [1]:
import os
import warnings
import torch

In [2]:
warnings.filterwarnings('ignore')

In [3]:
workdir, _ = os.path.split(os.path.abspath(os.getcwd()))
input = os.path.join(os.getcwd(), 'SepReformer/sample_wav/sample_WSJ.wav')

In [4]:
from simple_vg.vg_types import AudioSetting

settings: AudioSetting = {
    'mono': True,
    'n_fft': None,
    'sr': 16000,
    'use_torch': True,
    'window_size': None,
    'opt_settings': None
}


In [5]:
from simple_vg.process_audio import LoadAudioFile, LoadAudioSettings, LoadAudioBackends

load_settings: LoadAudioSettings = {
    'backend': LoadAudioBackends.TORCH_SOX,
    'effects': [],
    'channel_first': True
}

settings['opt_settings'] = load_settings

load_audio = LoadAudioFile(settings)

In [6]:
from simple_vg.simple_elements import ClosureElement

only_wav = ClosureElement(lambda _, inp: inp[0], None)

In [7]:
from simple_vg.speech_separation.sepreformer import *

sep_model_settings: SepReformerSetting = {
    'batch_size': 1,
    'chunk_max_len': None,
    'distributed_gpu': False,
    'model': SepReformerModels.SepReformer_Base_WSJ0,
    'n_speaker': 2,
    'sr': 16000
}


In [8]:
from simple_vg.process_overlap import *

sep_settings: SeparateOverlappedSettings = {
    'model_settings': sep_model_settings,
    'model': SeparateOverlappedModels.SepReformer
}

overlap_settings: ProcessOverlappedAudioSettings = {
    'tasks': ProcessOverlappedAudioTasks.SEPARATE_AUDIO,
    'separation_settings': sep_settings
}

settings['opt_settings'] = overlap_settings
overlap_elem = ProcessOverlappedAudio(settings)

In [9]:
from simple_vg.activity_detection import *

vad_p_settings: VADPipelineSetting = {
    'vad_model': VADModels.Silero_VAD
}

vad_settings: VoiceActivityDetectionSetting = {
    'pipeline_settings': vad_p_settings,
    'tasks': [
        VoiceActivityDetectionTasks.ACTIVITY_DETECTION, 
        VoiceActivityDetectionTasks.SPLIT_BY_ACTIVITY
    ]
}

settings['opt_settings'] = vad_settings

vad_elem = VoiceActivityDetection(settings)

In [10]:
import simple_vg.extract_text as stt
from simple_vg.extract_text import ExtractText

stt_settings = stt.default_settings()
stt_settings['batch_size'] = 2
settings['opt_settings'] = stt_settings
stt_mod = ExtractText(settings) #Input shape: (..., S) [(x, y, z, w, S) => view(x*y*z*w, S)]


In [11]:
import sys
from simple_vg.pipeline import VGPipeline

pipeline_elems = [
    load_audio, 
    only_wav, 
    overlap_elem,
    stt_mod]
pipeline = VGPipeline()
pipeline.sequential(pipeline_elems)
pipeline.run(input=input, dbg=True)


Index-0 LoadAudioFile Pipeline Output: tuple, shape: None
Index-1 ClosureElement Pipeline Output: Tensor, shape: torch.Size([1, 147186])
Index-2 ProcessOverlappedAudio Pipeline Output: Tensor, shape: torch.Size([1, 2, 1, 147184])
torch.Size([2, 147184])
torch.Size([2, 128, 919])
[[DecodingResult(audio_features=tensor([[ 0.0354,  0.3909, -0.3801,  ..., -0.0731, -0.6230,  0.1898],
        [-0.0543, -0.0077, -0.1112,  ...,  0.2563,  0.6362, -0.2537],
        [-0.2637, -0.1633, -0.1600,  ...,  0.0516,  0.6382,  0.6450],
        ...,
        [-0.1323, -0.3552, -0.0682,  ..., -0.1740, -0.0274,  0.0084],
        [-0.2705, -0.0249, -0.4829,  ..., -0.3511, -0.2399, -0.3486],
        [-0.2588,  0.0562, -0.5127,  ..., -0.6357,  0.0512,  0.2532]],
       device='cuda:0', dtype=torch.float16), language='en', language_probs=None, tokens=[50365, 4162, 264, 624, 13, 50, 13, 13346, 575, 38942, 42893, 294, 5162, 3259, 11, 415, 8194, 572, 643, 337, 264, 12380, 26049, 281, 5300, 1179, 6846, 281, 2371, 264

In [12]:
stt_mod.model.dataset.indice

[[0, 0, 919], [1, 0, 919]]

In [13]:
stt_mod.model.result

[[DecodingResult(audio_features=tensor([[ 0.0354,  0.3909, -0.3801,  ..., -0.0731, -0.6230,  0.1898],
          [-0.0543, -0.0077, -0.1112,  ...,  0.2563,  0.6362, -0.2537],
          [-0.2637, -0.1633, -0.1600,  ...,  0.0516,  0.6382,  0.6450],
          ...,
          [-0.1323, -0.3552, -0.0682,  ..., -0.1740, -0.0274,  0.0084],
          [-0.2705, -0.0249, -0.4829,  ..., -0.3511, -0.2399, -0.3486],
          [-0.2588,  0.0562, -0.5127,  ..., -0.6357,  0.0512,  0.2532]],
         device='cuda:0', dtype=torch.float16), language='en', language_probs=None, tokens=[50365, 4162, 264, 624, 13, 50, 13, 13346, 575, 38942, 42893, 294, 5162, 3259, 11, 415, 8194, 572, 643, 337, 264, 12380, 26049, 281, 5300, 1179, 6846, 281, 2371, 264, 7241, 13, 50824], text='Since the U.S. currency has strengthened sharply in recent weeks, he sees no need for the Federal Reserve to raise interest rates to protect the dollar.', avg_logprob=-0.03788858301499311, no_speech_prob=1.1595845811340766e-12, temperature=